# View Exposures BG40


- work with Weakly_2022_09
- use jupyter kernel LSST



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/05/30



In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
import astropy.io.fits as pyfits

# be sure all these libraries are installed
pip install holoviews
pip install xarray


In [ ]:
# Bokeh for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize
from holoviews.operation.datashader import datashade, dynspread
from holoviews.plotting.util import process_cmap

import datashader as dsh

In [ ]:
# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
#hv.extension('bokeh')
#hv.extension('bokeh', 'matplotlib')
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

In [ ]:
#import lsst.afw.display as afwDisplay
#afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
import lsst.daf.butler as dafButler

# Functions

In [ ]:
def parse_slice_str(s):
    """
    Parse slicing string like [2345:1234,567:890] and returns
    the four values in the same order as 2 tuples, eg:
    parse_slice_str('[2345:1234,567:890]') -> (2345,1234),(567,890).
    """
    r = s.replace('[','').replace(']','').strip() # remove brackets

    parts = r.split(',')
    if len(parts) != 2:
        raise ValueError("Invalid format for slice '%s'" % s)

    subparts_first = parts[0].split(':')
    if len(subparts_first) != 2:
        raise ValueError("Invalid format for slice '%s' (first slice)" % s)
    first_values = (int(subparts_first[0]), int(subparts_first[1])) # may raise ValueError, ok

    subparts_second = parts[1].split(':')
    if len(subparts_second) != 2:
        raise ValueError("Invalid format for slice '%s' (second slice)" % s)
    second_values = (int(subparts_second[0]), int(subparts_second[1])) # may raise ValueError, ok

    return first_values, second_values

# Configuration

## Select flags options

In [ ]:
FLAG_ROTATE_IMG = False
FLAG_TRANSFORM = False

## Transformations
astropy scale transformations

In [ ]:
#transform = AsinhStretch() + PercentileInterval(99.)
transform = PercentileInterval(98.)

## Range in image flat values

In [ ]:
VMIN = 0.5
VMAX = 1.5

## Histograms bins

In [ ]:
NBINS_HISTO = 200

## Holoview Elements config

### Histogram config

In [ ]:
HV_HISTO_SINGLE_WIDTH  = 400
HV_HISTO_SINGLE_HEIGHT = 350
HV_HISTO_MULTI_WIDTH  = 350
HV_HISTO_MULTI_HEIGHT = 300
HV_HISTO_MULTI_COLS   = 3

## Image configuration

In [ ]:
HV_IMAGE_SINGLE_WIDTH  = 400
HV_IMAGE_SINGLE_HEIGHT = 400
HV_IMAGE_SINGLE_FRAME_WIDTH = 600
HV_IMAGE_MULTI_WIDTH  = 400
HV_IMAGE_MULTI_HEIGHT = 400
HV_IMAGE_MULTI_FRAME_WIDTH = 350
HV_IMAGE_MULTI_COLS   = 3

## Butler

In [ ]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

## Utils

In [ ]:
def isflat(row):
    #print(row["filter"])
    if str(row["type"]) == "flat":
        return True
    
    else:
        return False 

In [ ]:
TOOLTIPS = [
    ('name', "$name"),
    ('index', "$index"),
    ('pattern', '@pattern'),
    ("x", "$x"),
    ("y", "$y"),
    ("value", "@image"),
    ('squared', '@squared')
]

TOOLTIPS = [
    ("(x,y)", "($x, $y)"),
]

hover = HoverTool(description='Custom Tooltip', tooltips=[('x', '@x'), ('y', '@y')])


# Custom hover tool for the source detections
myhover = HoverTool(
    tooltips=[
        ( 'x', '@x{0.2f}'),
        ( 'y', '@y{0.2f}'),
    ],
    formatters={
        'x' : 'printf',
        'y' : 'printf',
    },
    
)

In [ ]:
# Define some default plot options for the Image
img_opts = dict(
                #height=600, width=700, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '8pt'},
                colorbar=True, toolbar='right', show_grid=True,
                aspect='equal',
                frame_width=HV_IMAGE_SINGLE_FRAME_WIDTH ,
                tools=['hover','crosshair','undo','redo','zoom_in','zoom_out'],
                #tools=[myhover,'crosshair'],
               )     

# Input BG40

In [ ]:
path_raw = "/sps/lsst/groups/auxtel/data/raw_ncsa/2021-11-04"

In [ ]:
files_raw = os.listdir(path_raw)
#files_raw

In [ ]:
seqnum = 508

In [ ]:
files_to_remove = []
for filename in files_raw:
    if not "508" in filename:
        files_to_remove.append(filename)

In [ ]:
for file_to_remove in files_to_remove:
    files_raw.remove(file_to_remove)

In [ ]:
files_raw

In [ ]:
indexfl    = 0
filename   = files_raw[indexfl]
fullfilename = os.path.join(path_raw,filename) 
thetitle = filename.split(".")[0] 

In [ ]:
filename 

# Stitch

In [ ]:
fitsname = fullfilename
fits = pyfits.open(fitsname)

primhdu = fits[0]
primhdr = primhdu.header

# DETSIZE = '[1:4072,1:4000]'    / Size of sensor                                 
# OVERV   =                   48 / Vert-overscan pix                              
# OVERH   =                   64 / Over-scan pixels                               
## The following is probably wrong, according to the sequencer code used
## for the readout of the ITL CCD and to the resulting geometry: 3 + 509 + 64 = 576
# PREH    =                    0 / Pre-scan pixels                                
## --> here we use instead PREH = 3

## Detector size (illuminated area) 

detsize_str = primhdr['DETSIZE']
(ymin_b1, ymax_b1), (xmin_b1, xmax_b1) = parse_slice_str(detsize_str) # 1-base coordinates
xmin, xmax = xmin_b1-1, xmax_b1-1  # 0-base coordinates
xsize = xmax - xmin + 1
ymin, ymax = ymin_b1-1, ymax_b1-1  # 0-base coordinates
ysize = ymax - ymin + 1

print("Detector size: ")
print("  xsize =", xsize)
print("  ysize =", ysize)

In [ ]:
## Prescan and Overscan

amp_prescan_lines    = 0
amp_overscan_lines   = primhdr['OVERV']
amp_prescan_columns  = 3  # primhdr['PREH']  # header is clearly inconsistent here
amp_overscan_columns = primhdr['OVERH']

## Create the large image

ccd = np.zeros(dtype = fits[1].data.dtype, shape = (xsize, ysize))

## Process the HDUs for the 16 amplifiers

for hdu in fits[1:]:
    hdr = hdu.header
    img = hdu.data

    amp_total_xsize, amp_total_ysize = img.shape
    amp_ill_xsize = amp_total_xsize - amp_prescan_lines - amp_overscan_lines
    amp_ill_ysize = amp_total_ysize - amp_prescan_columns - amp_overscan_columns

    
    print("Processing HDU <%s> ..." % hdr['EXTNAME'])

    print("  HDU img.shape = ", img.shape)
    print("  HDU ill_xsize = ", amp_ill_xsize)
    print("  HDU ill_ysize = ", amp_ill_ysize)

    ## illuminated part in the amplifier data...
    # datasec_str = hdu.header['DATASEC']
    # datasec_slice = parse_slice_str(datasec_str)
    # in fact the header DATASEC is wrong.

    amp_ill_xmin = amp_prescan_lines
    amp_ill_xmax = amp_prescan_lines + amp_ill_xsize

    amp_ill_ymin = amp_prescan_columns
    amp_ill_ymax = amp_prescan_columns + amp_ill_ysize

    
    print("  illum area [0-based python-style] = [%d:%d,%d:%d]" % 
          ( amp_ill_xmin, amp_ill_xmax, amp_ill_ymin, amp_ill_ymax ))

    illum = img[amp_ill_xmin:amp_ill_xmax, amp_ill_ymin:amp_ill_ymax]
    print("  illum.shape = ", illum.shape)
    
    ## ... and where it sits in the physical CCD array.
    detsec_str = hdu.header['DETSEC']
    detsec_slice = parse_slice_str(detsec_str)
    # Loading also the transfo vector and matrix (1-based)
    # DTV1    =                  510 / detector transformation vector                 
    # DTV2    =                    0 / detector transformation vector                 
    # DTM1_1  =                  -1. / detector transformation matrix                 
    # DTM2_2  =                   1. / detector transformation matrix                 
    # DTM1_2  =                    0 / detector transformation matrix                 
    # DTM2_1  =                    0 / detector transformation matrix                 
    
    dtv1 = hdr['DTV1']; dtv2 = hdr['DTV2']
    dtm1_1 = hdr['DTM1_1']; dtm1_2 = hdr['DTM1_2']
    dtm2_1 = hdr['DTM2_1']; dtm2_2 = hdr['DTM2_2']

    print("  dtv = ", np.array([dtv1, dtv2]))
    print("  dtm = ")
    print(np.array([[dtm1_1, dtm1_2],[dtm2_1, dtm2_2]]))
    
    ccd_x1 = detsec_slice[1][0]
    ccd_x2 = detsec_slice[1][1]
    # ccd_xstep = int(dtm2_2)
    if ccd_x1 < ccd_x2: 
        ccd_xstep = 1
    else:
        ccd_x1, ccd_x2 = ccd_x2, ccd_x1
        ccd_xstep = -1
    #
    ccd_x1 -= 1

    ccd_y1 = detsec_slice[0][0]
    ccd_y2 = detsec_slice[0][1]
    # ccd_ystep = int(dtm1_1)
    if ccd_y1 < ccd_y2:
        ccd_ystep = 1
    else:
        ccd_y1, ccd_y2 = ccd_y2, ccd_y1
        ccd_ystep = -1
    #
    ccd_y1 -= 1
        
    # print("  DATASEC = ", datasec_slice)
    print("  DETSEC = ", detsec_slice)
    
    print("  CCD area [0-based python-style] = [%d:%d,%d:%d]" % 
          (ccd_x1, ccd_x2, ccd_y1, ccd_y2) )

    # print(ccd[ccd_x1:ccd_x2:ccd_xstep, ccd_y1:ccd_y2:ccd_ystep].shape)
    print(ccd[ccd_x1:ccd_x2, ccd_y1:ccd_y2].shape)

    ccd[ccd_x1:ccd_x2, ccd_y1:ccd_y2] = illum[::ccd_xstep, ::ccd_ystep]


In [ ]:
# Save the resulting array in a new FITS file

outfitsname = filename.replace(".fits", "-stitched.fits")
outfitshdr = primhdr
pyfits.writeto(outfitsname, ccd, header=primhdr, overwrite=True)

In [ ]:
ccd.shape

In [ ]:
img_data = ccd

In [ ]:
img_data.shape

In [ ]:
if FLAG_TRANSFORM: 
    scaledImage = transform(img_data)
else:
    scaledImage = img_data

In [ ]:
if FLAG_ROTATE_IMG:
    rotscaledImage=np.rot90(scaledImage)
else:
    rotscaledImage = scaledImage

In [ ]:
rotscaledImage

In [ ]:
plt.imshow(rotscaledImage)

In [ ]:
VMIN= img_data.ravel().min()
VMAX= img_data.ravel().max()

In [ ]:
hv.help(hv.Image)

In [ ]:
# Define some default plot options for the Image
img_opts = dict(
                height=700, width=700, 
                xaxis="bottom", 
                fontsize={'ticks':20, 'title':20,'ylabel':18, 'xlabel':18, 'zlabel':5,'legend':8, 'legend_title':13},
                colorbar=True, show_grid=True,
                aspect='equal',
                #fig_size=HV_IMAGE_SINGLE_FRAME_WIDTH ,
                tools=['hover','crosshair','undo','redo','zoom_in','zoom_out'],
                #tools=[myhover,'crosshair'],
               )    

In [ ]:
# Create the Image element.
bounds_img=(0,0,rotscaledImage.shape[1],rotscaledImage.shape[0])
img = hv.Image(rotscaledImage, bounds=bounds_img,
               kdims=['x', 'y']).opts(
    cmap = "Greys_r",  xlabel = 'x', ylabel ='y',
    title = thetitle)

In [ ]:
img

In [ ]:
rasterize(img)